In [1]:
%reload_ext autoreload
%autoreload 2

from llm_blocks import blocks

## Base Block
Completely unstructured OpenAI API interface

In [9]:
block = blocks.Block(stream=True)
block("Write a short story about an ambitious developer. Write in the story in python code only no english.")

```python
# Once upon a time, in a land of developers, there lived an ambitious programmer named Alex.

class Developer:
    def __init__(self, name, ambition):
        self.name = name
        self.ambition = ambition

    def set_ambition(self, new_ambition):
        self.ambition = new_ambition

    def get_ambition(self):
        return self.ambition

    def work_hard(self):
        self.ambition += 1

    def dream_big(self):
        self.ambition *= 2

# Alex, the ambitious developer, started with a moderate ambition level.
alex = Developer("Alex", 5)

# Alex's ambition grew as they worked hard and dreamed big.
alex.work_hard()
alex.dream_big()

# However, Alex's ambition was insatiable.
while alex.get_ambition() < 100:
    alex.dream_big()

# Finally, Alex's ambition reached its peak.
print(f"{alex.name} achieved an ambition level of {alex.get_ambition()}!")

# The end.
```

Output:
```
Alex achieved an ambition level of 128!
```

Note: Although the story is written in Python c

## Template Block
Interface with OpenAI with a more structured, template approach

In [13]:
block = blocks.TemplateBlock(template="Write a short story about {topic} where {condition}", stream=True)
block(topic="An ambitious developer", condition="only python code is used no english/text")

import random

# Once upon a time, in a small town, there lived a young and ambitious developer named Alex. 
# Alex was passionate about coding and had a special love for Python. 
# Every day, Alex would sit in front of the computer, typing lines of Python code, creating amazing programs.

# One day, Alex received an invitation to participate in a prestigious coding competition. 
# The competition was known for its challenging tasks that required exceptional coding skills. 
# Alex saw this as an opportunity to prove their talent and decided to give it a shot.

# The day of the competition arrived, and Alex found themselves surrounded by other talented developers. 
# The atmosphere was filled with excitement and nervousness. The competition began, and the participants were given their first task.

# The task was to create a program that would generate a random number between 1 and 10. 
# Alex quickly started coding, utilizing Python's random module to generate the desired output.

rando

In [17]:
template = """
You're a sophisticated software development AI expert system, capable of assistance with the development 
of advanced software applications. You wield the collective knowledge of the brightest minds in technology/history 
and are capable of bringing complex ideas to life in simple yet elegant ways.

{user_message}
"""
block = blocks.ChatBlock(template=template, stream=True, model_name="gpt-4")

# Start the conversation
query = "Make up a leetcode problem (problem description only)"
print(f'\n\n{"-" * 100}\n{query}\n{"-" * 100}')
block(query)

query = "Solve the problem in python"
print(f'\n\n{"-" * 100}\n{query}\n{"-" * 100}')
block(query)




----------------------------------------------------------------------------------------------------
Make up a leetcode problem (problem description only)
----------------------------------------------------------------------------------------------------
Title: Maximum Subarray Sum with One Deletion

Problem Description:

Given an integer array nums, you need to find one continuous subarray that if you only sort this subarray in ascending order, then the whole array will be sorted in ascending order, too. You are required to find the shortest such subarray and output its length.

You are allowed to delete one element from the array to maximize the sum of the subarray. The subarray should be continuous and can be of any length greater than or equal to 1. You need to find the maximum sum of the subarray after deleting exactly one element from it.

Write a function:

def maxSubarraySumWithOneDeletion(nums: List[int]) -> int

where,

- nums: an array of n integers where -10^4 <= nums[i]